# Twitter API testing

In [9]:
import tweepy
import time

In [10]:
CONSUMER_KEY = 'P8uR3oU2eTF1hZw2SX1lma8Zw'
CONSUMER_SECRET = 'Kg85hTJdliuTdFroydQvoRsg7cpr5WZm2MdEQtCz8EjcjG04dp'
ACCESS_TOKEN = '386339280-5sy1Smvnkw91cBObIwu3ju6aMQJU4B8X8HZNsDSo'
ACCESS_TOKEN_SECRET = 'WuZNLBRfzZgT6DIDnZOPqaVlt8uMtrvOUJ9aSbUwkmocD'

auth = tweepy.OAuth1UserHandler(
  CONSUMER_KEY, 
  CONSUMER_SECRET, 
  ACCESS_TOKEN, 
  ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth, wait_on_rate_limit=True)

### Data we need to retrieve given a user profile

1. Number of followers
2. Number of original tweets and number of retweets (last 7 days)

For each original tweet:

3. Number of retweets from other users (shares)
4. Number of favorites from other users (likes)
5. Tweet text
6. Replies from other users to user tweet (text)

7. Gather all the data for future SPI computation

We will take Elon Musk user "@JFCadavid" as an example

The API has some limitations:
* We will only be able to retrieve tweets from the last 7 days.
* There is a rate limit of 60req/min. Going beyond will break the Twitter API (i.e. will block temporarily our access tokens)

Having 'RT @user:' as the beginning of a tweet, tells me that this is a retweet

1. Get number of followers

In [11]:
n_followers = api.search_users("JFCadavid")[0].followers_count

2. Get the number of tweets (Including retweets, last 7 days)

In [12]:
extracted_tweets = []

for status in tweepy.Cursor(api.search_tweets, 
                            "from:JFCadavid",
                            count=5).items(100):
    extracted_tweets.append(status)

In [13]:
retweets = [tweet for tweet in extracted_tweets if "RT @" in tweet.text]
tweets = set(extracted_tweets) - set(retweets)
n_retweets = len(retweets)
n_tweets = len(tweets)

print(n_retweets, n_tweets)

5 5


For each original tweet: 3. Get the number of retweets from another users, 4. get the number of favorites from other users (likes) and 5. get the tweet's text

In [14]:
# We will also store the id
n_id_ls = []
n_retweets_ls = []
n_favorites_ls = []
tweet_text_ls = []
for tweet in tweets:
    n_id_ls.append(tweet.id)
    n_retweets_ls.append(tweet.retweet_count)
    n_favorites_ls.append(tweet.favorite_count)
    tweet_text_ls.append(tweet.text)

In [15]:
tweet_text_ls

['¡EN VIVO TRAS LA GOLEADA DE INGLATERRA ANTE IRÁN!\n\n¿Carlos Queiroz es mal técnico?\n\n¿Los británicos son favoritos a… https://t.co/gyXyIBrkHB',
 '@ProfeVictorHAC Yo respondí siguiendo la mamadera de gallo',
 '¡EN VIVO TRAS LA PRIMERA CAÍDA, EN LA HISTORIA, DE UN PAÍS SEDE EN SU PARTIDO DEBUT!\n\nEl VAR: ¿Buen o mal protagoni… https://t.co/gVNvcY0m51',
 '@ProfeVictorHAC Pero sin cigarrillo, con toda la ropa y a las 5 am de Qatar tras trabajar tooooooodo el día.',
 'Países Bajos 2-0 Senegal\n\n¿Esta naranja es mecánica o una del montón?\n\n¿Esperaban más de un equipo africano o es lo… https://t.co/vk7w75FDUj']

5. Get the replies from other users to user tweets

In [16]:
def get_replies(user_name, tweet_id, max_replies=10):
    replies = tweepy.Cursor(api.search_tweets, q='to:{}'.format(user_name),
                                    since_id=tweet_id, tweet_mode='extended').items()

    replies_ls = []

    counter = 0
    while counter < max_replies :
        try:
            reply = replies.next()
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if reply.in_reply_to_status_id == tweet_id:
                replies_ls.append(reply.full_text)
                counter = counter + 1 

        except StopIteration:
            break

        except Exception as e:
            print("Failed while fetching replies {}".format(e))
            break
    return replies_ls

In [17]:
replies_ls = [get_replies("JFCadavid", tweet_id) for tweet_id in n_id_ls]

In [18]:
tweet_text_ls

['¡EN VIVO TRAS LA GOLEADA DE INGLATERRA ANTE IRÁN!\n\n¿Carlos Queiroz es mal técnico?\n\n¿Los británicos son favoritos a… https://t.co/gyXyIBrkHB',
 '@ProfeVictorHAC Yo respondí siguiendo la mamadera de gallo',
 '¡EN VIVO TRAS LA PRIMERA CAÍDA, EN LA HISTORIA, DE UN PAÍS SEDE EN SU PARTIDO DEBUT!\n\nEl VAR: ¿Buen o mal protagoni… https://t.co/gVNvcY0m51',
 '@ProfeVictorHAC Pero sin cigarrillo, con toda la ropa y a las 5 am de Qatar tras trabajar tooooooodo el día.',
 'Países Bajos 2-0 Senegal\n\n¿Esta naranja es mecánica o una del montón?\n\n¿Esperaban más de un equipo africano o es lo… https://t.co/vk7w75FDUj']

In [19]:
replies_ls

[['@JFCadavid Irán tiene su James...',
  '@JFCadavid Si. No. No me interesa.',
  '@JFCadavid Inglaterra vs Tahremi',
  '@JFCadavid No-no',
  '@JFCadavid Siempre entran de favoritos y se desinflan en los partidos con Selecciónes top. Inglaterra, Países Bajos, Bélgica, España',
  '@JFCadavid Eso fue que le hicieron el cajón los Iraníes a Queiroz…..'],
 [],
 ['@JFCadavid En Aras de la transparencia en este partido con el tema de haber comprado el resultado a favor del anfitrión. Queda claro que solo fue expeculasion. Bien por el fútbol y en especial por Ecuador.',
  '@JFCadavid Fuera de lugar no sé dónde. Las imágenes del Var confunden aun más.'],
 ['@JFCadavid @ProfeVictorHAC Y espero que sin tatuajes en las nalgas! 😂 La verdad que comparación tan absurda "señor" Victor',
  '@JFCadavid @ProfeVictorHAC A eso los llevaron no? A trabajar, si le cuesta tanto regrésese hombre JF.',
  '@JFCadavid @ProfeVictorHAC Y cuál es el problema con que hagas videos desde la cama?',
  '@JFCadavid @ProfeVi

7. Gather all the data for future SPI computation

In [25]:
from datetime import date
import numpy as np

user1_data = {
    "user_name": "JFCadavid",
    "n_followers": n_followers,
    "n_retweets": n_retweets,
    "n_tweets": n_tweets,
    "n_retweets_to_user": np.sum(n_retweets_ls),
    "n_favorites_to_user": np.sum(n_favorites_ls),
    "n_replies_to_user": np.sum([len(replies_post) for replies_post in replies_ls]),
    "tweets_text": tweet_text_ls,
    "tweets_replies": replies_ls,
    "date": date.today().strftime("%d/%m/%Y")
}

In [144]:
date.today().strftime("%d/%m/%Y")

'18/11/2022'

In [149]:
import json
with open("sample_data.json", "w") as outfile:
    json.dump(user1_data, outfile)

In [24]:
user1_data

{'user_name': 'JFCadavid',
 'n_followers': 628287,
 'n_retweets': 5,
 'n_tweets': 5,
 'n_retweets_to_user': 3,
 'n_favorites_to_user': 423,
 'n_replies': 23,
 'tweets_text': ['¡EN VIVO TRAS LA GOLEADA DE INGLATERRA ANTE IRÁN!\n\n¿Carlos Queiroz es mal técnico?\n\n¿Los británicos son favoritos a… https://t.co/gyXyIBrkHB',
  '@ProfeVictorHAC Yo respondí siguiendo la mamadera de gallo',
  '¡EN VIVO TRAS LA PRIMERA CAÍDA, EN LA HISTORIA, DE UN PAÍS SEDE EN SU PARTIDO DEBUT!\n\nEl VAR: ¿Buen o mal protagoni… https://t.co/gVNvcY0m51',
  '@ProfeVictorHAC Pero sin cigarrillo, con toda la ropa y a las 5 am de Qatar tras trabajar tooooooodo el día.',
  'Países Bajos 2-0 Senegal\n\n¿Esta naranja es mecánica o una del montón?\n\n¿Esperaban más de un equipo africano o es lo… https://t.co/vk7w75FDUj'],
 'tweets_replies': [['@JFCadavid Irán tiene su James...',
   '@JFCadavid Si. No. No me interesa.',
   '@JFCadavid Inglaterra vs Tahremi',
   '@JFCadavid No-no',
   '@JFCadavid Siempre entran de favori